# 🎓 Travaux Pratiques : DCGAN (Deep Convolutional GAN)

**Auteur :**Benlahmar Habib


## Objectifs Pédagogiques

1.  **Architecture Convolutive :** Comprendre comment les couches `Conv2d` et `ConvTranspose2d` améliorent la qualité des images générées par rapport au MLP-GAN.
2.  **Stabilité de l'Entraînement :** Assimiler le rôle critique de la **Normalisation par Batch (Batch Normalization)** et l'ajustement de la normalisation des données à $[-1, 1]$.
3.  **Implémentation DCGAN :** Mettre en œuvre le modèle DCGAN original sur Fashion-MNIST.

--- 

## I. Fondements du DCGAN

Le DCGAN est l'un des premiers modèles à établir des lignes directrices claires pour construire des GANs stables et performants avec des réseaux profonds et convolutifs (Radford et al., 2016).

### 1.1. Lignes Directrices Architecturales

1.  **Remplacer les couches de *Pooling*** : Utiliser des convolutions à **pas (stride)** fractionné (`ConvTranspose2d`) dans $G$ et des convolutions à pas dans $D$ pour les opérations d'échantillonnage (upsampling/downsampling).
2.  **Normalisation par Batch (BN) :** Appliquer `nn.BatchNorm2d` au Générateur et au Discriminateur pour stabiliser les activations et prévenir le gradient vanishing (sauf la première couche de $D$ et la couche de sortie de $G$).
3.  **Activations :** Utiliser **ReLU** dans $G$ (sauf la sortie) et **LeakyReLU** dans $D$ pour permettre aux gradients de circuler sans problème.
4.  **Sortie :** Utiliser **`nn.Tanh()`** dans la couche de sortie de $G$ pour forcer les pixels dans l'intervalle $[-1, 1]$.

### Question  (Q1.1)

Pourquoi la *Normalisation par Batch (BatchNorm)* est-elle particulièrement bénéfique dans l'entraînement du Discriminateur, qui est déjà un classifieur binaire standard ? (Indice : Pensez à l'échelle des activations et à la lutte entre $G$ et $D$).

---

## II. Configuration et Préparation des Données à $[-1, 1]$

**ATTENTION :** En raison de l'activation `nn.Tanh()` dans le Générateur, nous devons normaliser les données d'entrée de Fashion-MNIST dans l'intervalle **$[-1, 1]$** (au lieu de $[0, 1]$). La formule utilisée est $\mathbf{x}' = (\mathbf{x} - 0.5) \times 2$.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
from torchvision.utils import make_grid
from tqdm.notebook import trange, tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Exécution sur {device}")

# Hyperparamètres
latent_dim = 100
batch_size = 128
epochs = 50
lr = 0.0002

# NORMALISATION CRITIQUE : [0, 1] -> [-1, 1]
transform_dcgan = transforms.Compose([
    transforms.ToTensor(), # Normalise en [0, 1]
    transforms.Normalize((0.5,), (0.5,)) # (x - 0.5) / 0.5 -> [-1, 1]
])

# Chargement des données
train_dataset = datasets.FashionMNIST(root='./data/FashionMNIST', train=True, download=True, transform=transform_dcgan)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialisation des poids (selon les recommandations DCGAN)
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        init.normal_(m.weight.data, 1.0, 0.02)
        init.constant_(m.bias.data, 0)
    elif classname.find('Linear') != -1:
        init.normal_(m.weight.data, 0.0, 0.02)
        init.constant_(m.bias.data, 0)


### Question  (Q2.1)

Expliquez pourquoi le Générateur doit utiliser l'activation `Tanh` en sortie, et comment cela contraint le prétraitement des données à une normalisation dans l'intervalle $[-1, 1]$.

--- 

## III. Architecture du DCGAN (28x28)

Nous ajustons l'architecture DCGAN pour la taille $28 \times 28$ de Fashion-MNIST. Les couches `ConvTranspose2d` et `Conv2d` sont l'analogue de *l'upsampling* et du *downsampling*.

### 3.1. Le Générateur ($G_{\text{DCGAN}}$)

Input : $\mathbf{z}$ (100 dimensions). Output : $1 \times 28 \times 28$. Les couches `ConvTranspose2d` augmentent la résolution spatiale.

In [ ]:
# Nombre de feature maps (canaux)
ngf = 64 # Nombre de 'Generative Feature maps'

class Generator(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.main = nn.Sequential(
            # Projection initiale du bruit z (100) en volume spatial (ngf*4 x 4 x 4)
            nn.ConvTranspose2d(latent_dim, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # État : (ngf*4) x 4 x 4

            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # État : (ngf*2) x 8 x 8 (résolution non standard 28x28, ajustement nécessaire)
            
            # Pour atteindre 28x28 à partir de 8x8, nous avons besoin d'un upsampling progressif
            # Simplification pour 28x28 (en ajustant le padding/kernel pour le dernier bloc)
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # État : (ngf) x 16 x 16

            # Dernière couche pour atteindre 28x28 (kernel 4, stride 2, padding 3)
            nn.ConvTranspose2d(ngf, 1, 4, 2, 3, bias=False), # Output: 28 x 28 x 1
            nn.Tanh() # Sortie normalisée entre [-1, 1]
        )

    def forward(self, z):
        # z doit être transformé en volume 4D pour les ConvTranspose2d
        return self.main(z.view(-1, latent_dim, 1, 1))


### 3.2. Le Discriminateur ($D_{\text{DCGAN}}$)

Input : $1 \times 28 \times 28$. Output : Logit scalaire. Les couches `Conv2d` réduisent la résolution spatiale. **NOTE** : Pas de BN sur la première couche de $D$ (selon les recommandations DCGAN).

In [ ]:
ndf = 64 # Nombre de 'Discriminator Feature maps'

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            # Couche d'entrée (pas de BN) : 1 x 28 x 28 -> ndf x 14 x 14
            nn.Conv2d(1, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # ndf x 14 x 14 -> ndf*2 x 7 x 7
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # ndf*2 x 7 x 7 -> ndf*4 x 4 x 4
            nn.Conv2d(ndf * 2, ndf * 4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # Couche de sortie : ndf*4 x 4 x 4 -> 1 (logit)
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False), 
            # La sortie n'est pas activée (pas de Sigmoid), elle est en format logit
        )

    def forward(self, input):
        # Le Discriminateur renvoie un logit (scalaire) après un squeeze()
        return self.main(input).view(-1, 1)


### Question  (Q3.1)

Observez l'architecture du Générateur. Quel est le rôle de la couche `ConvTranspose2d` dans ce réseau ? Comment le fait d'utiliser des convolutions, plutôt que des couches linéaires, permet-il de garantir une meilleure **cohérence spatiale** (c'est-à-dire des formes plus réalistes) dans les images générées ?

--- 

## IV. Entraînement et Stabilité

La boucle d'entraînement Minimax reste la même, mais nous utilisons la nouvelle architecture et l'initialisation des poids spécialisée.

In [ ]:
# Initialisation des modèles et application des poids
G = Generator(latent_dim).to(device).apply(weights_init)
D = Discriminator().to(device).apply(weights_init)

# Optimiseurs (Adam avec betas ajustés)
G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
D_optimizer = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

criterion = nn.BCEWithLogitsLoss() 

# Bruit fixe pour la visualisation
fixed_noise = torch.randn(64, latent_dim, device=device)

# Fonction utilitaire de visualisation (adaptée pour [-1, 1] -> [0, 1] pour matplotlib)
def show_grid(grid, title="", figsize=(10, 10)):
    plt.figure(figsize=figsize)
    plt.title(title)
    # Inverse la normalisation pour l'affichage : [-1, 1] -> [0, 1]
    grid = (grid + 1) / 2
    plt.imshow(np.transpose(grid.numpy(), (1, 2, 0)), cmap="gray")
    plt.axis("off")
    plt.show()

def train_dcgan(G, D, G_optimizer, D_optimizer, criterion, dataloader, epochs, latent_dim, device):
    
    for epoch in trange(epochs, desc="Entraînement DCGAN"):
        for i, (real_images, _) in enumerate(dataloader):
            
            # Les images sont déjà en [-1, 1] et en format 4D (batch, 1, 28, 28)
            real_images = real_images.to(device)
            b_size = real_images.size(0)
            
            # Cibles pour BCEWithLogitsLoss
            real_labels = torch.full((b_size, 1), 1.0, device=device)
            fake_labels = torch.full((b_size, 1), 0.0, device=device)

            # ===============================================
            # 1. Mise à jour du Discriminateur (D)
            # ===============================================
            D.zero_grad()

            # 1a. Loss VRAI
            output = D(real_images)
            D_loss_real = criterion(output, real_labels)

            # 1b. Loss FAUX
            noise = torch.randn(b_size, latent_dim, device=device)
            fake = G(noise)
            output = D(fake.detach()) # D.detach()!
            D_loss_fake = criterion(output, fake_labels)

            # 1c. Rétropropagation D
            D_loss = D_loss_real + D_loss_fake
            D_loss.backward()
            D_optimizer.step()

            # ===============================================
            # 2. Mise à jour du Générateur (G)
            # ===============================================
            G.zero_grad()
            
            # G veut que D classe les fausses images comme VRAIES (cible = 1)
            output = D(fake) 
            G_loss = criterion(output, real_labels) 
            
            G_loss.backward()
            G_optimizer.step()
            
        tqdm.write(f"Epoch {epoch+1:2d} | D Loss: {D_loss.item():.4f} | G Loss: {G_loss.item():.4f}")
        
        # 3. Visualisation
        if (epoch + 1) % 5 == 0:
            G.eval()
            with torch.no_grad():
                generated_images = G(fixed_noise).cpu()
                show_grid(make_grid(generated_images, 8), title=f"DCGAN Génération Époque {epoch+1}")
            G.train()

# train_dcgan(G, D, G_optimizer, D_optimizer, criterion, train_dataloader, epochs, latent_dim, device) # <-- DÉCOMMENTER POUR LANCER L'ENTRAINEMENT

### Question  (Q4.1)

Quelle est la fonction de `weights_init(m)` ? Pourquoi est-il considéré comme une **bonne pratique** d'initialiser les poids des couches convolutionnelles avec une distribution Gaussienne centrée à $0$ avec un petit écart-type ($0.02$) dans les DCGANs, plutôt que d'utiliser l'initialisation par défaut de PyTorch ?

--- 

## V. Synthèse et Ouverture (Post-Entraînement)

Analysez les résultats de votre DCGAN après l'entraînement.

### Questions Finales

1.  **Stabilité :** Après l'entraînement, la perte de $D$ (D Loss) est-elle proche de 0.69 ? Comparez la stabilité de l'entraînement de ce DCGAN à celle du MLP-GAN précédent. Comment la Normalisation par Batch a-t-elle potentiellement contribué à cette différence ?
2.  **Qualité d'Image :** Décrivez la différence de qualité des images générées par le DCGAN par rapport au MLP-GAN. Quelles structures sont mieux capturées grâce à l'utilisation des convolutions ?
3.  **Défis :** Bien que les DCGANs soient plus stables, ils sont toujours sujets au **Mode Collapse**. Décrivez brièvement comment l'architecture DCGAN tente *indirectement* de le prévenir (Indice : diversité des cartes de caractéristiques).
4.  **Ouverture (WGAN) :** Le prochain défi majeur des GANs concerne le choix de la fonction de coût. Citez le problème principal de la divergence de Jensen-Shannon (utilisée par BCE) lorsque les distributions réelles et générées sont séparées. Quel modèle de GAN célèbre (ex: WGAN) tente de résoudre ce problème en utilisant une distance alternative (la distance de Wasserstein) ? 